In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)
from src.extractor_utils import (predict, getPrediction,save_prediction,
                                 load_prediction, torch_summarize,RandomSearch)

from utils.utils import (create_submission ,metrics2csv)

# Making results reproducible

In [2]:
torch.backends.cudnn.deterministic = True
seed=1719
np.random.seed(seed)
torch.manual_seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 

# Checking for resources avaliables

In [3]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G        146G         15G        1.3G         58G         70G
Swap:            0B          0B          0B
Sun Dec  3 14:45:06 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   42C    P0    74W / 149W |   2439MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [4]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [5]:
use_gpu = True
use_DataParalel= True
use_CPU= False       

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS



# Path to features

In [6]:
path2features= "./features/" 

# RandomSearch on hyperparameters

In [7]:
# Flag to optimze hyperparameters
hiper_tunning = True

# Choosing Model

In [8]:
use_resnet = False
use_inception = False
use_denseNet = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()  

Using DenseNet model


In [9]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        model.mrnc = model.mrnc.cuda()
        model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        model.mrnd = model.mrnd.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
else:
    print("Using CPU's")

Using all GPU's 


# Loading features

In [10]:
model_name

'DenseNet'

In [11]:
load_feat= load_prediction(path2features,model_name)

Loaded features with shapes: 


train:
pred torch.Size([17940, 2208]), true torch.Size([17940])

valid:
pred torch.Size([4484, 2208]), true torch.Size([4484])

test:
pred torch.Size([79726, 2208]), true torch.Size([79726])


In [12]:
load_feat['train'][0].shape,load_feat['train'][1].shape

(torch.Size([17940, 2208]), torch.Size([17940]))

In [13]:
conv_dset ={
'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
}

# Creating Dataloaders

In [14]:
batch_size= 32

dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [15]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 2492, 'train': 561, 'valid': 141}

# Trainning Model

In [16]:
if hiper_tunning: 
    model_name += 'RandomSearch' 

path2saveModel = './models/'+model_name+'DistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=0)

In [17]:
num_epochs = 3

loss_fn = nn.CrossEntropyLoss()
optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3,weight_decay=0)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric()] #ptt.PlotCallback(),
}

In [18]:
# Search space for tunning hyperparameters 
args= {'lr':[1e-5,1e-2],
       'weight_decay':[1e-8,1e-3] }

In [19]:
if hiper_tunning: 

    output = RandomSearch(params,args,
                          num_epochs=num_epochs,path2saveModel=path2saveModel,
                          dset_loaders_convnet=dset_loaders_convnet,
                          MAX_IT=2,verbose=2)
    
    trainer = output['best_trainer']
    print('****************************')
    print('\nBest parameters {}'.format(output['best_parameters']))
    print('\nBest result {}'.format(output['best_result']))

else:
    params['callbacks'].append(ptt.PrintCallback())
    scheduler = StepLR(optimizer, step_size=5, gamma=0.55)
    trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,lr_scheduler=scheduler,**params)
    trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Iteration 0/2
Start training for 3 epochs
  1:  16.3s   T: 1.19204 0.58556   V: 0.57462 0.80932 best
  2:   8.7s   T: 0.43312 0.85892   V: 0.30484 0.90299 best
  3:   8.5s   T: 0.33185 0.89766   V: 0.46505 0.86084 
Stop training at epoch: 3/3
evaluate: 560/560 ok
evaluate: 140/140 ok
train_loss: 0.2200937603163214, val_loss 0.3048385311512943
Execution time :37.074058055877686 s
lr: 0.004513174892036614, weight_decay: 0.00012929722630595036
Iteration 1/2
Start training for 3 epochs
  1:   8.6s   T: 0.67803 0.82029   V: 0.52030 0.84746 best
  2:   8.7s   T: 0.46857 0.86488   V: 0.62695 0.81958 
  3:   8.7s   T: 0.45293 0.87475   V: 0.46276 0.88359 best
Stop training at epoch: 3/3
evaluate: 560/560 ok
evaluate: 140/140 ok
train_loss: 0.32733059151382615, val_loss 0.4627550516034959
Execution time :29.719228982925415 s
lr: 0.009697252441171573, weight_decay: 0.0002340885167516301
****************************

Best parameters {'lr': 0.0045131748920366139, 'weight_decay': 0.0001292972263059

In [38]:
#print(torch_summarize(model))

# Loading best model

In [39]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [40]:
metrics2csv(trainer, model_name)

done!


In [41]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

In [42]:
train_eval, valid_eval

({'losses': 0.2200937590770498}, {'losses': 0.30483853226788443})

# Dataset without shuffling 

#### Utilized to plot mismatch cases

In [25]:
path2train = "/mnt/home/e209440/data/train"  

In [26]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}   

In [27]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [28]:
dsets = {
    'valid': KaggleSafeDriverDataset(path2train,
                                     transforms=data_transforms['valid'],
                                     use_only=use_only, is_val=True, val_size=0.2),
}

In [29]:

dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size,
                                           pin_memory=False, use_shuffle= False)

# Evaluating resultings 

In [30]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_convnet['train'], model.mrnd, use_gpu=use_gpu)

result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)

result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)

predict: 560/560 ok
predict: 140/140 ok
predict: 2491/2491 ok


In [31]:
predictions_out = {'train': (result_train['pred'], result_train['true'], model_name),
                   'valid': (result_valid['pred'], result_valid['true'], model_name),
                   'test': (result_test['pred'], result_test['true'], model_name)}

In [32]:
path2results = './results/'

In [33]:
save_prediction(path2results,predictions_out)


Saving DenseNetRandomSearch train
Saved in:./results/train/DenseNetRandomSearch.npz

Saving DenseNetRandomSearch valid
Saved in:./results/valid/DenseNetRandomSearch.npz

Saving DenseNetRandomSearch test
Saved in:./results/test/DenseNetRandomSearch.npz


In [34]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [35]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [36]:
print('Train: ', correct_train, '/', '17940' )# len(dsets['train'])
print('Valid: ', correct_valid, '/', '4484' ) # len(dsets['valid'])

Train:  16643 / 17940
Valid:  3824 / 4484


# Make submission of the Test set

In [37]:
create_submission(result_test, model_name + '_distracted_driver')

done!
